# Knight move on keypad

## Import functions

In [ ]:
import sys, os

# Add the parent directory to the sys.path
if "__file__" in globals().keys():
    sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
else:
    sys.path.append(os.path.dirname(os.getcwd()))

In [ ]:
from src.table_data import get_table_data
from src.positions import get_vowel_positions, get_missing_positions, get_coordinate_bounds
from src.path_check import check_inside_bounds, check_inside_bounds, check_in_avoided_coordinate
from src.path_create import get_all_paths
from src.vowel_check import has_at_most_two_vowels

## Initial tests

In [ ]:
# Create the main coordinate data containing the letter/number grid
# This creates coordinate tuples in format (row, col, value) for a 4x5 grid
coordinate_data = get_table_data()
print("Keypad (Coordinate Data):")
print("First few coordinates:")
for i, coord in enumerate(coordinate_data[:10]):
    print(f"  {coord}")
print(f"  ... and {len(coordinate_data) - 10} more coordinates")
print()

# Find all missing/empty positions in the coordinate data
# These positions will be avoided during path generation
miss_pos = get_missing_positions(coordinate_data)
print(f"Missing positions (to avoid): {miss_pos}")

# Find all vowel positions (A, E, I, O, U) in the coordinate data  
# These are special positions that we want to limit visits to
vo_pos = get_vowel_positions(coordinate_data)
print(f"Vowel positions (limited visits): {vo_pos}")

# Extract the numeric bounds of the coordinate data
# This gives us [x_min, x_max, y_min, y_max] for boundary checking
bounds_dict = get_coordinate_bounds(coordinate_data)
idx_bounds = [bounds_dict['x_min'], bounds_dict['x_max'], bounds_dict['y_min'], bounds_dict['y_max']]
print(f"Index bounds [x_min, x_max, y_min, y_max]: {idx_bounds}")
print()

In [ ]:
# Create a filter function that validates knight move paths
# A valid path must:
# 1. Stay within the coordinate boundaries (check_inside_bounds)
# 2. Avoid all missing/empty positions (not check_in_avoided_coordinate)
filter = lambda path: (check_inside_bounds(path, *idx_bounds)) and (not check_in_avoided_coordinate(path, miss_pos))

In [ ]:
# Test paths
paths = get_all_paths(1, 1, 3, location_list=vo_pos, max_hit=2,
               filter=filter,
               )

print(f"Total number of paths: {len(paths)}")
print("Paths:")

# Create a lookup dictionary for coordinate values
coord_lookup = {(x, y): value for x, y, value in coordinate_data}

for path in paths:
    path_string = ("".join([coord_lookup[i, j] for i, j in path]))
    print(path)
    print(path_string)

In [ ]:
# Check the total number of paths

x_i = 1
y_i = 1
step_i = 3

print("Create all paths then apply filter:")

all_path_list = get_all_paths(x_i, y_i, step_i, location_list=vo_pos, max_hit=2)

print(f"Total number of paths inside and outside the bounds: {len([path for path in all_path_list if not check_inside_bounds(path, *idx_bounds)]) + len([path for path in all_path_list if check_inside_bounds(path, *idx_bounds)])}")

print(f"Total number of paths hitting or not hitting the missing positions: {len([path for path in all_path_list if not check_in_avoided_coordinate(path, miss_pos)]) + len([path for path in all_path_list if check_in_avoided_coordinate(path, miss_pos)])}")

path_list_after_filter = [path for path in all_path_list if filter(path)]

print(f"Total number of paths after applying the filter: {len(path_list_after_filter)}")

print("Create all paths with filter:")

all_path_list = get_all_paths(x_i, y_i, step_i, location_list=vo_pos, max_hit=2, filter=filter)

print(f"Total number of paths inside and outside the bounds: {len([path for path in all_path_list if not check_inside_bounds(path, *idx_bounds)]) + len([path for path in all_path_list if check_inside_bounds(path, *idx_bounds)])}")

print(f"Total number of paths hitting or not hitting the missing positions: {len([path for path in all_path_list if not check_in_avoided_coordinate(path, miss_pos)]) + len([path for path in all_path_list if check_in_avoided_coordinate(path, miss_pos)])}")

path_list_with_filter = [path for path in all_path_list if filter(path)]

print(f"Total number of paths: {len(path_list_with_filter)}")

# Create a lookup dictionary for coordinate values
coord_lookup = {(x, y): value for x, y, value in coordinate_data}

# Transform each path to path_string
print("\nPath strings from path_list_with_filter:")
for i, path in enumerate(path_list_with_filter):
    path_string = "".join([coord_lookup[i, j] for i, j in path])
    print(f"Path {i+1}: {path} -> '{path_string}'")

## Efficiency analysis

In [ ]:
import time
from datetime import datetime

def time_execution(func, *args, description="Function execution", **kwargs):
    """
    Time the execution of a function and return results with timing information.
    
    Parameters:
    -----------
    func : callable
        The function to time
    *args : tuple
        Positional arguments for the function
    description : str
        Description of what's being timed
    **kwargs : dict
        Keyword arguments for the function
    
    Returns:
    --------
    tuple
        (result, execution_time_seconds)
    """
    print(f"\n🕐 Starting: {description}")
    print(f"📅 Start time: {datetime.now().strftime('%H:%M:%S.%f')[:-3]}")
    
    start_time = time.perf_counter()
    result = func(*args, **kwargs)
    end_time = time.perf_counter()
    
    execution_time = end_time - start_time
    
    print(f"⏱️  Execution time: {execution_time:.4f} seconds")
    print(f"📅 End time: {datetime.now().strftime('%H:%M:%S.%f')[:-3]}")
    
    if execution_time < 0.001:
        print(f"⚡ Very fast: {execution_time*1000:.2f} milliseconds")
    elif execution_time < 1:
        print(f"🏃 Fast: {execution_time*1000:.1f} milliseconds") 
    elif execution_time < 10:
        print(f"🚶 Moderate: {execution_time:.2f} seconds")
    else:
        print(f"🐌 Slow: {execution_time:.1f} seconds")
    
    return result, execution_time

# Time the path generation for different steps
print("="*60)
print("TIMING ANALYSIS FOR PATH GENERATION")
print("="*60)

step_i = 3
x_i, y_i = 1, 1
max_hit = 2

# Time different operations
timing_results = {}

# 1. Time basic path generation
paths, time1 = time_execution(
    get_all_paths, x_i, y_i, step_i,
    description=f"Basic path generation (step {step_i})"
)
timing_results['basic_paths'] = (len(paths), time1)

# 2. Time path generation with vowel location constraints
paths_vowel, time2 = time_execution(
    get_all_paths, x_i, y_i, step_i,
    location_list=vo_pos, max_hit=max_hit,
    description=f"Path generation with vowel constraints (step {step_i})"
)
timing_results['vowel_constrained'] = (len(paths_vowel), time2)

# 3. Time path generation with filter
paths_filtered, time3 = time_execution(
    get_all_paths, x_i, y_i, step_i,
    location_list=vo_pos, max_hit=max_hit, filter=filter,
    description=f"Path generation with filter (step {step_i})"
)
timing_results['filtered'] = (len(paths_filtered), time3)

# 4. Time the vowel checking loop
def vowel_check_loop():
    total_paths = 0
    # Create a lookup dictionary for coordinate values
    coord_lookup = {(x, y): value for x, y, value in coordinate_data}
    
    for x_i in range(1, 5):
        for y_i in range(1, 6):
            all_path_list = get_all_paths(x_i, y_i, step_i, location_list=vo_pos, max_hit=max_hit, filter=filter)
            for path in all_path_list:
                path_string = "".join([coord_lookup[i, j] for i, j in path])
                if has_at_most_two_vowels(path_string):
                    total_paths += 1
    return total_paths

vowel_count, time4 = time_execution(
    vowel_check_loop,
    description=f"Complete vowel analysis loop (step {step_i})"
)
timing_results['vowel_analysis'] = (vowel_count, time4)

# Display summary
print("\n" + "="*60)
print("TIMING SUMMARY")
print("="*60)
print(f"{'Operation':<35} {'Paths':<10} {'Time (s)':<12} {'Paths/sec':<12}")
print("-" * 60)

for operation, (path_count, exec_time) in timing_results.items():
    rate = path_count / exec_time if exec_time > 0 else 0
    print(f"{operation:<35} {path_count:<10} {exec_time:<12.4f} {rate:<12.1f}")

print("\n📊 Performance Insights:")
print(f"• Filter reduces paths by {((timing_results['basic_paths'][0] - timing_results['filtered'][0]) / timing_results['basic_paths'][0] * 100):.1f}%")
print(f"• Vowel constraints reduce paths by {((timing_results['basic_paths'][0] - timing_results['vowel_constrained'][0]) / timing_results['basic_paths'][0] * 100):.1f}%")
print(f"• Processing rate: {timing_results['vowel_analysis'][0] / timing_results['vowel_analysis'][1]:.1f} vowel checks per second")

## Test different methods to get the paths

In [ ]:
import time

x_min, x_max = bounds_dict['x_min'], bounds_dict['x_max']
y_min, y_max = bounds_dict['y_min'], bounds_dict['y_max']
max_hit = 2

### Apply filter when creating the paths

In [ ]:
# Iterate through different path lengths and total_paths total paths with timing

print("Path Analysis with Timing:")
print("=" * 60)
print(f"{'Step':<6} {'Path Length':<12} {'Total Paths':<12} {'Time (s)':<10} {'Paths/sec':<10}")
print("-" * 60)

for step_i in [3, 5, 7, 9]:
    start_time = time.perf_counter()
    
    total_paths = 0
    for x_i in range(x_min, x_max + 1):
        for y_i in range(y_min, y_max + 1):
            all_path_list = get_all_paths(x_i, y_i, step_i, location_list=vo_pos, max_hit=max_hit, filter=filter)
            total_paths += len(all_path_list)
    
    end_time = time.perf_counter()
    execution_time = end_time - start_time
    path_length = step_i + 1  # step_i is number of moves, path length is moves + 1
    paths_per_sec = total_paths / execution_time if execution_time > 0 else 0
   
    print(f"{step_i:<6} {path_length:<12} {total_paths:<12} {execution_time:<10.4f} {paths_per_sec:<10.1f}")

print("=" * 60)

### Apply filter after creating the paths

In [ ]:
# Iterate through different path lengths with vowel checking and timing

print("Path Analysis with Vowel Checking and Timing:")
print("=" * 70)
print(f"{'Step':<6} {'Path Length':<12} {'Total Paths':<12} {'Time (s)':<10} {'Paths/sec':<10}")
print("-" * 70)

# Create a lookup dictionary for coordinate values (once, outside the loop)
coord_lookup = {(x, y): value for x, y, value in coordinate_data}

for step_i in [3, 5, 7, 9]:
    start_time = time.perf_counter()
    
    total_paths = 0
    for x_i in range(x_min, x_max + 1):
        for y_i in range(y_min, y_max + 1):

            all_path_list = get_all_paths(x_i, y_i, step_i)
            path_list_final = [path for path in all_path_list if filter(path)]

            for path in path_list_final:
                path_string = ("".join([coord_lookup[i, j] for i, j in path]))
                
                if has_at_most_two_vowels(path_string):
                    total_paths += 1

    end_time = time.perf_counter()
    execution_time = end_time - start_time
    path_length = step_i + 1  # step_i is number of moves, path length is moves + 1
    paths_per_sec = total_paths / execution_time if execution_time > 0 else 0
    
    print(f"{step_i:<6} {path_length:<12} {total_paths:<12} {execution_time:<10.4f} {paths_per_sec:<10.1f}")

print("=" * 70)